In [1]:
import pandas as pd
import tensorflow as tf
from sklearn import metrics
import numpy as np

In [2]:
class_labels = ['Professor Name', 'Office Hours', 'Office Location', 'Lecture Timings', 'Lecture Location', 'Class Location']
dataframe = pd.read_table("professorName.txt", header=None)
dataframe['class'] = 0
train_data = dataframe
#print train_data

In [3]:
dataframe = pd.read_table("officeHours.txt", header=None)
dataframe['class'] = 1
train_data = train_data.append(dataframe)

In [4]:
dataframe = pd.read_table("officeLocation.txt", header=None)
dataframe['class'] = 2
train_data = train_data.append(dataframe)

In [5]:
dataframe = pd.read_table("lectureTimings.txt", header=None)
dataframe['class'] = 3
train_data = train_data.append(dataframe)

In [6]:
dataframe = pd.read_table("lectureLocation.txt", header=None)
dataframe['class'] = 4
train_data = train_data.append(dataframe)

In [7]:
dataframe = pd.read_table("classLocation.txt", header=None)
dataframe['class'] = 5
train_data = train_data.append(dataframe)

In [8]:
print train_data

                                                  0  class
0                                    name professor      0
1                                     name lecturer      0
2                                   name instructor      0
3                                      name teacher      0
4                                name the professor      0
5                                 name the lecturer      0
6                               name the instructor      0
7                                  name the teacher      0
8                                 name of professor      0
9                                  name of lecturer      0
10                               name of instructor      0
11                                  name of teacher      0
12                            name of the professor      0
13                             name of the lecturer      0
14                           name of the instructor      0
15                              name of the teacher     

In [25]:
x_train = np.array(train_data[0])
y_train = np.array(train_data['class'])
x_test = np.array(train_data[0])
y_test = np.array(train_data['class'])
print x_train
y_train = np.array(y_train).reshape([-1, 1])
print y_train

['name professor' 'name lecturer' 'name instructor' ...,
 'where is the class located' 'where is the classroom location'
 'where is the classroom located']
[[0]
 [0]
 [0]
 ..., 
 [5]
 [5]
 [5]]


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit(x_train)
x_train_dtm = vect.transform(x_train)
x_test_dtm = vect.transform(x_test)
input_size =  len(vect.get_feature_names())
#print x_train_dtm
simple_train_dtm = vect.transform(x_train)
#simple_train_dtm
#pd.DataFrame(simple_train_dtm.toarray(), columns=vect.get_feature_names())
x_cnn = simple_train_dtm.toarray()
print x_cnn[0]
y_cnn = np.array(tf.one_hot(y_train,6))
print y_cnn

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0]
Tensor("one_hot:0", shape=(4064, 6), dtype=float32)


In [11]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [12]:
nb.fit(x_train_dtm, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [13]:
y_pred_class = nb.predict(x_test_dtm)
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)
metrics.confusion_matrix(y_test, y_pred_class)

array([[  88,    0,    0,    0,    8,    0],
       [   0, 1032,    0,    0,    0,    0],
       [   0,   15,  295,    0,    0,    0],
       [   0,    0,    0,  900,    0,    0],
       [   0,    0,    0,   15, 1695,    0],
       [   0,    0,    0,    0,    8,    8]])

In [14]:
print class_labels[np.asscalar(nb.predict(x_test_dtm[0]))]

Professor Name


In [15]:
query = ["syllabus of class"]
msg = vect.transform(query)
print msg

  (0, 4)	1
  (0, 19)	1


In [16]:
print class_labels[np.asscalar(nb.predict(msg))]

Lecture Timings


In [17]:
# input placeholders
X = tf.placeholder(tf.float32, [None, input_size])
X_img = tf.reshape(X, [-1,1,input_size,1]) 
y = tf.placeholder(tf.int32, [None, 1])
Y = tf.one_hot(y,6)
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1 = tf.nn.conv2d(X_img, W1,strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1,ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L2 = tf.reshape(L2, [-1, 7 * 7 * 64])
W3 = tf.get_variable("W3", shape=[7 * 7 * 64, 6], initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([6]))
hypothesis = tf.matmul(L2, W3) + b
# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)

In [32]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# train my model
print('Learning stared. It takes sometime.')
for epoch in range(10):
    avg_cost = 0
    #total_batch = int(mnist.train.num_examples / batch_size)   
    #for i in range(4064):
    #batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    for i in range(4064):
        feed_dict = {X: np.array(x_cnn[i]).reshape([1, 32]), y: np.array(y_train[i]).reshape([-1,1])}
        c, _, = sess.run([cost, optimizer],feed_dict=feed_dict)       
        avg_cost += c / 4064
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(c))

Learning stared. It takes sometime.


InvalidArgumentError: Input to reshape is a tensor with 512 values, but the requested shape requires a multiple of 3136
	 [[Node: Reshape_1 = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/gpu:0"](MaxPool_1, Reshape_1/shape)]]
	 [[Node: Mean/_15 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_273_Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'Reshape_1', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658,